In [1]:
import cv2
import time

eye_cascade_path = 'haarcascade_eye_tree_eyeglasses.xml'
face_cascade_path = 'haarcascade_frontalface_alt.xml'

face_cascade = cv2.CascadeClassifier(face_cascade_path)
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

cap = cv2.VideoCapture(0)  

drowsy_start_time = 0
drowsy_state = False
drowsy_alert_triggered = False

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture frame from the video source.")
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        (x, y, w, h) = faces[0]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        face_roi = frame[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        if len(eyes) == 0:
            if not drowsy_state:
                drowsy_start_time = time.time()
                drowsy_state = True
            cv2.putText(frame, "!!", (460, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 0, 255), 2)
        else:
            if drowsy_state:
                drowsy_state = False
            cv2.putText(frame, "Non Drowsy!!", (460, 50), cv2.FONT_HERSHEY_COMPLEX, 0.9, (0, 255, 0), 2)

        if drowsy_state and time.time() - drowsy_start_time >= 1 and not drowsy_alert_triggered:
            cv2.putText(frame, "DROWSY ALERT!", (260, 100), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 255), 3)
            drowsy_alert_triggered = True

        resized_face = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_LINEAR)
        cv2.imshow('Face Recognition', resized_face)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
